<a href="https://colab.research.google.com/github/keerthanab2201/Sentiment-Analysis-using-Deep-Learning/blob/main/tcn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Collection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load csv file and preview
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/datasets/Amazon-Product-Reviews-Sentiment-Analysis-in-Python-Dataset.csv")
print(df.head())

                                              Review  Sentiment
0  Fast shipping but this product is very cheaply...          1
1  This case takes so long to ship and it's not e...          1
2  Good for not droids. Not good for iPhones. You...          1
3  The cable was not compatible between my macboo...          1
4  The case is nice but did not have a glow light...          1


In [3]:
# Save as a JSON file(records format)
df.to_json("amazon_reviews_data.json", orient="records", lines=True)
print("✅ Conversion complete: Saved as reviews_data.json")

✅ Conversion complete: Saved as reviews_data.json


##Data Pre-Processing

* lowercase
* stopword removal
* punctuation removal
* one word review removal
* contraction removal
* tokenization
* part of speech tagging

In [4]:
# installing dependencies
!pip install contractions textblob gensim beautifulsoup4
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# Step 2: Import Modules
import re
import pandas as pd
import nltk
import numpy as np
import gensim
from bs4 import BeautifulSoup
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from contractions import fix as expand_contractions
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
# load JSON dataset and inspect columns
df = pd.read_json("amazon_reviews_data.json", lines=True)
print(df.columns)

Index(['Review', 'Sentiment'], dtype='object')


In [8]:
# Drop missing values and filter required columns
text_col = "Review"
label_col = "Sentiment"
df = df[[text_col, label_col]].dropna() #these are the two columns
df.columns = ["text", "rating"]  # Normalize column names

In [9]:
# Define preprocessing function
def preprocess_pipeline(text):
    text = str(text)
    text = expand_contractions(text.lower())            # Lowercase + contractions
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)          # Elongated word normalization
    tokens = word_tokenize(text)                        # Tokenization
    # Optional: Stopwords (skip for deep models)
    # tokens = [t for t in tokens if t not in stop_words]
    tokens = [re.sub(r"[^\w\s!?]", "", t) for t in tokens]  # 5. Remove selective punctuation
    tokens = [t for t in tokens if t.strip() != ""]
    if len(tokens) <= 1:
        return None
    pos_tags = nltk.pos_tag(tokens)
    clean_text = " ".join(tokens)
    polarity_score = TextBlob(clean_text).sentiment.polarity
    return {
        "clean_text": clean_text,
        "tokens": tokens,
        "pos_tags": pos_tags,
        "score": polarity_score
    }


In [10]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [11]:
# Apply preprocessing function
processed = df["text"].apply(preprocess_pipeline)
df = df[processed.notnull()].copy()
df["processed"] = processed[processed.notnull()].values

In [12]:
# Extract cleaned data for tokenization
texts = df["processed"].apply(lambda x: x["clean_text"]).tolist()
labels = df["rating"].tolist()
scores = df["processed"].apply(lambda x: x["score"]).tolist()
# Result: Three lists containing the text data, labels, and scores respectively

In [13]:
# Word Embeddings
# Keras Tokenizer- converts raw text into numerical sequences (each word= unique integer index) that can be later processed by Keras layers like Embedding
MAX_VOCAB = 10000
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="")
tokenizer.fit_on_texts(texts)
# sequence padding- adding placeholder values (often zeros) to shorter sequences in a dataset to make them all the same length
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post')

In [14]:
import zipfile

with zipfile.ZipFile("/content/drive/MyDrive/glove.6B.zip", "r") as zip_ref:
    zip_ref.extractall("glove")

In [15]:
vocab_size= 15000
# Load GloVe and Create Embedding Matrix
#GloVe(Global Vectors for Word Representation)- converts words into numerical vectors(embeddings) that capture semantic relationships between words- unsupervised learning algorithm

embedding_index = {}
with open("glove/glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

embedding_dim = 100  # matching model spec
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [16]:
# Remove neutral reviews and relabel
df = df[df['rating'] != 3]
df['label'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0) # creates binary labels: 1 for positive (rating ≥4), 0 for negative

/tmp/ipython-input-16-3232871527.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0) # creates binary labels: 1 for positive (rating ≥4), 0 for negative


In [17]:
# Get cleaned texts again
texts = df["processed"].apply(lambda x: x["clean_text"]).tolist()
labels = df["label"].tolist()

In [18]:
# Prepare tokenizer and sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [19]:
vocab_size = 15000
input_length = 400
# creates vocabulary of 15,000 most frequent words, converts text to integer sequences and pads sequences to length 400 (truncating longer texts or padding shorter ones)

In [20]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
padded_sequences = pad_sequences(sequences, maxlen=input_length, padding='post')

word_index = tokenizer.word_index

In [21]:
# setup embedding matrix
# initializes embedding matrix with zeros and fill it with pre-trained GloVe vectors where available
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if i >= vocab_size:
        continue
    vec = embedding_index.get(word)
    if vec is not None:
        embedding_matrix[i] = vec

In [22]:
# train test split- 70% training / 10% validation / 20% testing
from sklearn.model_selection import train_test_split

# Step 1: Split 20% test set
X_temp, X_test, y_temp, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42, stratify=labels
)

# Step 2: Split remaining 80% into 70% train and 10% validation
# 70% of full data = 87.5% of remaining 80%
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp
)

print(f"Train size     : {len(X_train)}")
print(f"Validation size: {len(X_val)}")
print(f"Test size      : {len(X_test)}")


Train size     : 13867
Validation size: 1981
Test size      : 3963


##TCN Model

In [23]:
!pip install keras-tcn
!pip install -q tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [35]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Add, BatchNormalization, Activation, Bidirectional, LSTM, GlobalMaxPooling1D, Conv1D
from tcn import TCN
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [26]:
# Hyperparameters
embedding_dim = 100
input_length = 400
num_filters = 128
kernel_size = 5
lstm_units = 64
dropout_rate = 0.5
l2_reg = 0.001

In [49]:
# 1. Enhanced TCN Block with Residual Connections
def TCN_block(x, filters, kernel_size, dilation_rate, l2_reg=1e-5):
    # Shortcut connection
    shortcut = x

    # Dilated convolution
    x = Conv1D(
        filters, kernel_size,
        padding='same',
        dilation_rate=dilation_rate,
        kernel_regularizer=l2(l2_reg)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)

    # Match dimensions for residual
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, padding='same')(shortcut)
    return Add()([x, shortcut])

# 2. Model Architecture
def build_tcn_model(input_length, vocab_size, embedding_dim, embedding_matrix):
    inputs = Input(shape=(input_length,))

    # Embedding Layer (fine-tuned)
    x = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=True
    )(inputs)

    # TCN Blocks (Fewer but Wider)
    x = TCN_block(x, 256, 3, 1)  # Increased filters to 256
    x = TCN_block(x, 256, 3, 2)
    x = GlobalMaxPooling1D()(x)

    # Classifier Head (Simplified)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    tcn_model = Model(inputs=inputs, outputs=outputs)

    # Optimizer with warmup (via ReduceLROnPlateau)
    optimizer = Adam(learning_rate=1e-3)

    tcn_model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return tcn_model

In [50]:
tcn_model = build_tcn_model(
    input_length=input_length,
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    embedding_matrix=embedding_matrix
)

In [51]:
# Callbacks
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mi2_lr=1e-6)
checkpoint = ModelCheckpoint("best_tcn_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)

In [52]:
# Model training
tcn_history = tcn_model.fit(
    X_train, np.array(y_train),
    validation_data=(X_val, np.array(y_val)),
    epochs=30,
    batch_size=64,
    callbacks=[early_stop, lr_scheduler, checkpoint],
    verbose=1,
    class_weight={0: 1, 1: 1}
)

Epoch 1/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6342 - loss: 1.4322
Epoch 1: val_accuracy improved from -inf to 0.82585, saving model to best_tcn_model.h5


217/217 ━━━━━━━━━━━━━━━━━━━━ 439s 2s/step - accuracy: 0.6346 - loss: 1.4290 - val_accuracy: 0.8258 - val_loss: 0.4842 - learning_rate: 0.0010
Epoch 2/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8305 - loss: 0.3798
Epoch 2: val_accuracy improved from 0.82585 to 0.86270, saving model to best_tcn_model.h5


217/217 ━━━━━━━━━━━━━━━━━━━━ 443s 2s/step - accuracy: 0.8306 - loss: 0.3798 - val_accuracy: 0.8627 - val_loss: 0.3587 - learning_rate: 0.0010
Epoch 3/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8740 - loss: 0.3061
Epoch 3: val_accuracy improved from 0.86270 to 0.87582, saving model to best_tcn_model.h5


217/217 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.8740 - loss: 0.3061 - val_accuracy: 0.8758 - val_loss: 0.3044 - learning_rate: 0.0010
Epoch 4/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9046 - loss: 0.2460
Epoch 4: val_accuracy did not improve from 0.87582
217/217 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.9046 - loss: 0.2460 - val_accuracy: 0.8693 - val_loss: 0.2971 - learning_rate: 0.0010
Epoch 5/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9232 - loss: 0.2028
Epoch 5: val_accuracy improved from 0.87582 to 0.88289, saving model to best_tcn_model.h5


217/217 ━━━━━━━━━━━━━━━━━━━━ 460s 2s/step - accuracy: 0.9232 - loss: 0.2028 - val_accuracy: 0.8829 - val_loss: 0.2794 - learning_rate: 0.0010
Epoch 6/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9407 - loss: 0.1625
Epoch 6: val_accuracy did not improve from 0.88289
217/217 ━━━━━━━━━━━━━━━━━━━━ 425s 2s/step - accuracy: 0.9407 - loss: 0.1626 - val_accuracy: 0.8788 - val_loss: 0.3104 - learning_rate: 0.0010
Epoch 7/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9558 - loss: 0.1301
Epoch 7: val_accuracy improved from 0.88289 to 0.88339, saving model to best_tcn_model.h5


217/217 ━━━━━━━━━━━━━━━━━━━━ 431s 2s/step - accuracy: 0.9558 - loss: 0.1301 - val_accuracy: 0.8834 - val_loss: 0.3646 - learning_rate: 0.0010
Epoch 8/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9660 - loss: 0.0979
Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 8: val_accuracy improved from 0.88339 to 0.88995, saving model to best_tcn_model.h5


217/217 ━━━━━━━━━━━━━━━━━━━━ 440s 2s/step - accuracy: 0.9660 - loss: 0.0979 - val_accuracy: 0.8900 - val_loss: 0.3432 - learning_rate: 0.0010
Epoch 9/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9762 - loss: 0.0714
Epoch 9: val_accuracy improved from 0.88995 to 0.89399, saving model to best_tcn_model.h5


217/217 ━━━━━━━━━━━━━━━━━━━━ 467s 2s/step - accuracy: 0.9762 - loss: 0.0714 - val_accuracy: 0.8940 - val_loss: 0.3155 - learning_rate: 5.0000e-04
Epoch 10/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9842 - loss: 0.0547
Epoch 10: val_accuracy did not improve from 0.89399
217/217 ━━━━━━━━━━━━━━━━━━━━ 424s 2s/step - accuracy: 0.9842 - loss: 0.0547 - val_accuracy: 0.8758 - val_loss: 0.4425 - learning_rate: 5.0000e-04
Epoch 11/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9845 - loss: 0.0542
Epoch 11: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 11: val_accuracy did not improve from 0.89399
217/217 ━━━━━━━━━━━━━━━━━━━━ 434s 2s/step - accuracy: 0.9845 - loss: 0.0542 - val_accuracy: 0.8910 - val_loss: 0.3883 - learning_rate: 5.0000e-04
Epoch 12/30
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9912 - loss: 0.0333
Epoch 12: val_accuracy did not improve from 0.89399
217/217 ━━━━━━━━━━━━━━━━━━━━ 444s 2s/step - accuracy: 0.9912 - loss: 0

KeyboardInterrupt: 

In [54]:
# Post-Training Boost (Test-Time Augmentation)
def predict_with_tta(model, X, n_samples=5):
    predictions = []
    for _ in range(n_samples):
        preds = tcn_model.predict(X, verbose=0).flatten()  # Enable dropout
        predictions.append(preds)
    return np.mean(predictions, axis=0)

y_pred_probs = predict_with_tta(tcn_model, X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

In [55]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predict
y_pred_prob = tcn_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print results
print("Confusion Matrix (TCN):")
print(cm)
print(f"\nAccuracy     : {accuracy:.4f}")
print(f"Precision    : {precision:.4f}")
print(f"Recall       : {recall:.4f}")
print(f"F1 Score     : {f1:.4f}")
print(f"Specificity  : {specificity:.4f}")

124/124 ━━━━━━━━━━━━━━━━━━━━ 24s 196ms/step
Confusion Matrix (TCN):
[[1795  198]
 [ 214 1756]]

Accuracy     : 0.8960
Precision    : 0.8987
Recall       : 0.8914
F1 Score     : 0.8950
Specificity  : 0.9007


In [57]:
import matplotlib.pyplot as plt

# plot accuracy and loss vs epoch
import matplotlib.pyplot as plt

# Plot Accuracy
plt.plot(tcn_history.history['accuracy'], label='Train Accuracy')
plt.plot(tcn_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy vs Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot Loss
plt.plot(tcn_history.history['loss'], label='Train Loss')
plt.plot(tcn_history.history['val_loss'], label='Validation Loss')
plt.title('Loss vs Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'tcn_history' is not defined